## Exercise: Heat transfer

This is a program that solves the heat transfer equation. It can be written as

$$df/dt = \Delta f$$

The initial condition is very simple. Everywhere inside the square the temperature equals f=0 and on the edges the temperature is f=x. This means the temperature goes from 0 to 1 in the direction of x. 

Our goal in this exercise is for you to parallelize the program with the OpenMP directives and clauses that you have learned so far.

In [ ]:
#pragma cling load("libomp.so")
#include <stdio.h>
#include <sys/time.h>
#include <omp.h>

// define functions MIN and MAX
#define min(A,B) ((A) < (B) ? (A) : (B))
#define max(A,B) ((A) > (B) ? (A) : (B))

// define size of grid points
#define imax 20
#define kmax 11
#define itmax 20000

// function prints the temperature grid, don't parallelize
void heatpr(double phi[imax+1][kmax+1]){
    int i,k,kl,kk,kkk;
    kl = 6; kkk = kl-1;
    for (k = 0; k <= kmax; k = k+kl){
        if(k+kkk > kmax) kkk = kmax-k;
        printf("\ncolumns %5d to %5d\n", k, k+kkk);
        for (i = 0; i <= imax; i++){
            printf("%5d ", i);
            for (kk = 0; kk <= kkk; kk++){
                printf("%#12.4g", phi[i][k+kk]);
            }
            printf("\n");
        }
    }
    return;
}

This is the main part of the code. In the time step iteration we calculate the temperature of the next time step. `dphi` is the difference of temperature and `phi` is the temperature. Then we add the `dphi` to the `phi` array and we save the new `phin` array. Then in the next for loop we exchange the role of the old and the new array (restoring the data). 

In [ ]:
// define variables
double eps = 1.0e-08;
double phi[imax+1][kmax+1], phin[imax][kmax];
double dx, dy, dx2, dy2, dx2i, dy2i, dt, dphi, dphimax, dphimax0;
int i, k, it;
struct timeval tv1, tv2; struct timezone tz;
double wt1, wt2;

printf("OpenMP-parallel with %1d threads\n", omp_get_num_threads());

dx = 1.0/kmax; dy = 1.0/imax;
dx2 = dx*dx; dy2 = dy*dy;
dx2i = 1.0/dx2; dy2i = 1.0/dy2;
dt = min(dx2,dy2)/4.0;

// setting initial conditions
/* start values 0.d0 */
for (i = 1; i < imax; i++){
    for (k = 0; k < kmax; k++){
        phi[i][k] = 0.0;
    }
}
/* start values 1.d0 */
for (i = 0;i <= imax; i++){
    phi[i][kmax] = 1.0;
}
/* start values dx */
phi[0][0] = 0.0;
phi[imax][0] = 0.0;
for (k = 1; k < kmax; k++){
    phi[0][k] = phi[0][k-1]+dx;
    phi[imax][k] = phi[imax][k-1]+dx;
}
// print starting values
printf("\nHeat Conduction 2d\n");
printf("\ndx = %12.4g, dy = %12.4g, dt = %12.4g, eps = %12.4g\n",
dx, dy, dt, eps);
printf("\nstart values\n");
heatpr(phi);

gettimeofday(&tv1, &tz);
wt1 = omp_get_wtime();
/* time step iteration */
for (it = 1; it <= itmax; it++){
    dphimax = 0.;
    dphimax0 = dphimax;
    for (k = 1; k < imax; k++){
        for (i = 0; i < kmax; i++){
            dphi = (phi[i+1][k]+phi[i-1][k]-2.*phi[i][k])*dy2i
                +(phi[i][k+1]+phi[i][k-1]-2.*phi[i][k])*dx2i;
            dphi = dphi*dt;
            dphimax0 = max(dphimax0,dphi);
            phin[i][k] = phi[i][k]+dphi;
        }
    }
    dphimax = max(dphimax,dphimax0);
    /* save values */
    for (i = 1; i < imax; i++){
        for (k = 0; k < kmax; k++){
            phi[i][k] = phin[i][k];
        }
    }
    if(dphimax < eps) break;
}
wt2 = omp_get_wtime();
gettimeofday(&tv2, &tz);

// print resulting grid and execution time
printf("\nphi after %d iterations\n", it);
heatpr(phi);
printf( "wall clock time (omp_get_wtime) = %12.4g sec\n", wt2-wt1 );
printf( "wall clock time (gettimeofday) = %12.4g sec\n", (tv2.tv_sec-tv1.tv_sec) + (tv2.tv_usec-tv1.tv_usec)*1e-6 );

You should parallelize the code above. Follow these tips and instructions. 
* TODO: parallelize all of the for loops
* TODO: use critical section for global maximum
* HINT: parallelize the outer loop because parallelizing the inner loop would give a parallelization overhead
* HINT: don't forget to make the inner loop index private
* Also think about other variables you might want to make private. 
* Don't forget to set the number of threads. 

Then run the example. Run it with 1, 2, 3, 4 threads and look at the execution time. 

You may see that with more threads it is slower than expected. Why is the parallel version significantly slower?

Compare your incomplete solution with this incomplete solution:

In [ ]:
// define variables
double eps = 1.0e-08;
double phi[imax+1][kmax+1], phin[imax][kmax];
double dx, dy, dx2, dy2, dx2i, dy2i, dt, dphi, dphimax, dphimax0;
int i, k, it;
struct timeval tv1, tv2; struct timezone tz;
double wt1, wt2;

int num_threads = 4;
omp_set_num_threads(num_threads);

# pragma omp parallel
{
    # pragma omp single
    printf("OpenMP-parallel with %1d threads\n", omp_get_num_threads());
}

dx = 1.0/kmax; dy = 1.0/imax;
dx2 = dx*dx; dy2 = dy*dy;
dx2i = 1.0/dx2; dy2i = 1.0/dy2;
dt = min(dx2,dy2)/4.0;

// setting initial conditions
#pragma omp parallel private(i,k) shared(phi)
{
    /* start values 0.d0 */
    #pragma omp for
    for (i = 1; i < imax; i++){
        for (k = 0; k < kmax; k++){
            phi[i][k] = 0.0;
        }
    }
    /* start values 1.d0 */
    #pragma omp for
    for (i = 0;i <= imax; i++){
        phi[i][kmax] = 1.0;
    }
}
/* start values dx */
phi[0][0] = 0.0;
phi[imax][0] = 0.0;
for (k = 1; k < kmax; k++){
    phi[0][k] = phi[0][k-1]+dx;
    phi[imax][k] = phi[imax][k-1]+dx;
}
// print starting values
printf("\nHeat Conduction 2d\n");
printf("\ndx = %12.4g, dy = %12.4g, dt = %12.4g, eps = %12.4g\n",
dx, dy, dt, eps);
printf("\nstart values\n");
heatpr(phi);

gettimeofday(&tv1, &tz);
wt1 = omp_get_wtime();
/* time step iteration */
for (it = 1; it <= itmax; it++){
    dphimax = 0.;
    #pragma omp parallel private(i,k,dphi,dphimax0) \
    shared(phi,phin,dx2i,dy2i,dt,dphimax)
    {
        dphimax0 = dphimax;
        #pragma omp for
        for (k = 1; k < imax; k++){
            for (i = 0; i < kmax; i++){
                dphi = (phi[i+1][k]+phi[i-1][k]-2.*phi[i][k])*dy2i
                    +(phi[i][k+1]+phi[i][k-1]-2.*phi[i][k])*dx2i;
                dphi = dphi*dt;
                dphimax0 = max(dphimax0,dphi);
                phin[i][k] = phi[i][k]+dphi;
            }
        }
        #pragma omp critical
        {
            dphimax = max(dphimax,dphimax0);
        }
        /* save values */
        #pragma omp for
        for (i = 1; i < imax; i++){
            for (k = 0; k < kmax; k++){
                phi[i][k] = phin[i][k];
            }
        }
    }
    if(dphimax < eps) break;
}
wt2 = omp_get_wtime();
gettimeofday(&tv2, &tz);

// print resulting grid and execution time
printf("\nphi after %d iterations\n", it);
heatpr(phi);
printf( "wall clock time (omp_get_wtime) = %12.4g sec\n", wt2-wt1 );
printf( "wall clock time (gettimeofday) = %12.4g sec\n", (tv2.tv_sec-tv1.tv_sec) + (tv2.tv_usec-tv1.tv_usec)*1e-6 );

Do you have any idea about why the parallel version is slower by looking at the code?

~~~c
for (k = 1; k < imax; k++){
    for (i = 0; i < kmax; i++){
        dphi = (phi[i+1][k]+phi[i-1][k]-2.*phi[i][k])*dy2i
            +(phi[i][k+1]+phi[i][k-1]-2.*phi[i][k])*dx2i;
        dphi = dphi*dt;
        dphimax0 = max(dphimax0,dphi);
        phin[i][k] = phi[i][k]+dphi;
    }
}
~~~

The sequence of the nested loops is wrong. In C/C++, the last array index is running the fastest so the `k` loop should be the inner loop. This is not fixed by the OpenMP compiler, so you will need to 
* TODO: interchange the sequence of the nested loops. 

Run the code again with 1, 2, 3, 4 threads and look at the execution time. 

Now the parallel version should be a little bit faster. The reason for only a slight improvement might be that the problem is too small and the parallelization overhead is too large. 

Compare your solution with this solution:

In [ ]:
// define variables
double eps = 1.0e-08;
double phi[imax+1][kmax+1], phin[imax][kmax];
double dx, dy, dx2, dy2, dx2i, dy2i, dt, dphi, dphimax, dphimax0;
int i, k, it;
struct timeval tv1, tv2; struct timezone tz;
double wt1, wt2;

int num_threads = 4;
omp_set_num_threads(num_threads);

# pragma omp parallel
{
    # pragma omp single
    printf("OpenMP-parallel with %1d threads\n", omp_get_num_threads());
}

dx = 1.0/kmax; dy = 1.0/imax;
dx2 = dx*dx; dy2 = dy*dy;
dx2i = 1.0/dx2; dy2i = 1.0/dy2;
dt = min(dx2,dy2)/4.0;

// setting initial conditions
#pragma omp parallel private(i,k) shared(phi)
{
    /* start values 0.d0 */
    #pragma omp for
    for (i = 1; i < imax; i++){
        for (k = 0; k < kmax; k++){
            phi[i][k] = 0.0;
        }
    }
    /* start values 1.d0 */
    #pragma omp for
    for (i = 0;i <= imax; i++){
        phi[i][kmax] = 1.0;
    }
}
/* start values dx */
phi[0][0] = 0.0;
phi[imax][0] = 0.0;
for (k = 1; k < kmax; k++){
    phi[0][k] = phi[0][k-1]+dx;
    phi[imax][k] = phi[imax][k-1]+dx;
}
// print starting values
printf("\nHeat Conduction 2d\n");
printf("\ndx = %12.4g, dy = %12.4g, dt = %12.4g, eps = %12.4g\n",
dx, dy, dt, eps);
printf("\nstart values\n");
heatpr(phi);

gettimeofday(&tv1, &tz);
wt1 = omp_get_wtime();
/* time step iteration */
for (it = 1; it <= itmax; it++){
    dphimax = 0.;
    #pragma omp parallel private(i,k,dphi,dphimax0) \
    shared(phi,phin,dx2i,dy2i,dt,dphimax)
    {
        dphimax0 = dphimax;
        #pragma omp for
        for (i = 1; i < imax; i++){
            for (k = 0; k < kmax; k++){
                dphi = (phi[i+1][k]+phi[i-1][k]-2.*phi[i][k])*dy2i
                    +(phi[i][k+1]+phi[i][k-1]-2.*phi[i][k])*dx2i;
                dphi = dphi*dt;
                dphimax0 = max(dphimax0,dphi);
                phin[i][k] = phi[i][k]+dphi;
            }
        }
        #pragma omp critical
        {
            dphimax = max(dphimax,dphimax0);
        }
        /* save values */
        #pragma omp for
        for (i = 1; i < imax; i++){
            for (k = 0; k < kmax; k++){
                phi[i][k] = phin[i][k];
            }
        }
    }
    if(dphimax < eps) break;
}
wt2 = omp_get_wtime();
gettimeofday(&tv2, &tz);

// print resulting grid and execution time
printf("\nphi after %d iterations\n", it);
heatpr(phi);
printf( "wall clock time (omp_get_wtime) = %12.4g sec\n", wt2-wt1 );
printf( "wall clock time (gettimeofday) = %12.4g sec\n", (tv2.tv_sec-tv1.tv_sec) + (tv2.tv_usec-tv1.tv_usec)*1e-6 );